# Export Results to Multiple Formats

In [1]:
# Loading all data and results
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os


In [2]:
# Loading the cleaned dataset
df_clean = pd.read_csv('../data/processed_data/clean_df.csv')

# Loading analysis results
with open('../data/processed_data/analysis_results.pkl', 'rb') as f:
    analysis_results = pickle.load(f)

# Extracting individual results
significance_results = analysis_results['significance_results']
correlation_df = analysis_results['correlation_df']
association_df = analysis_results['association_df']
coefficients = analysis_results['coefficients']
domains = analysis_results['domains']

print("All data loaded successfully")
print(f"Dataset shape: {df_clean.shape}")

All data loaded successfully
Dataset shape: (99, 41)


In [3]:
# Export functions for reports
def export_to_reports(data, filename, folder=''):
    """Export data to reports folder"""
    if folder:
        path = f'../reports/{folder}/{filename}'
    else:
        path = f'../reports/{filename}'
    
    if isinstance(data, pd.DataFrame):
        data.to_csv(path, index=False)
    elif isinstance(data, plt.Figure):
        data.savefig(path, dpi=300, bbox_inches='tight')
        plt.close()
    
    print(f"Exported: {path}")

# Exporting statistical results
export_to_reports(significance_results, 'significance_results.csv')
export_to_reports(correlation_df, 'correlation_analysis.csv')
export_to_reports(association_df, 'association_analysis.csv')
export_to_reports(coefficients, 'regression_coefficients.csv')
export_to_reports(df_clean, 'final_dataset.csv')

print("All statistical reports exported successfully to csv files.")

Exported: ../reports/significance_results.csv
Exported: ../reports/correlation_analysis.csv
Exported: ../reports/association_analysis.csv
Exported: ../reports/regression_coefficients.csv
Exported: ../reports/final_dataset.csv
All statistical reports exported successfully to csv files.


# Exporting Visualizations

In [4]:
# Creating and exporting visualizations
fig1 = plt.figure(figsize=(10, 6))
sns.heatmap(df_clean[['Daily_Screen_Time'] + domains].corr(), 
            annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix: Screen Time vs Health Impact Domains')
plt.tight_layout()
export_to_reports(fig1, 'correlation_matrix.png', 'figures')


Exported: ../reports/figures/correlation_matrix.png


In [5]:
# Exporting demographic distribution plots
fig2, axes = plt.subplots(2, 3, figsize=(15, 10))
fig2.suptitle('Screen Time Distribution by Demographic Variables')

variables = ['Age_Group', 'Gender_Group', 'Income_Group', 'Family_Type_Group', 'Devices_Group', 'Study_Hours_Group']
titles = ['Age Group', 'Gender', 'Annual Income', 'Family Type', 'Devices Owned', 'Study Hours']

for i, (var, title) in enumerate(zip(variables, titles)):
    row, col = i // 3, i % 3
    sns.countplot(data=df_clean, x=var, hue='Screen_Time_Group', ax=axes[row, col])
    axes[row, col].set_title(f'Screen Time by {title}')
    axes[row, col].tick_params(axis='x', rotation=45)

plt.tight_layout()
export_to_reports(fig2, 'demographic_distributions.png', 'figures')
plt.close()
print("Visualizations exported as PNG")

Exported: ../reports/figures/demographic_distributions.png
Visualizations exported as PNG


# Exporting to Excel (All results in one file)

In [6]:
with pd.ExcelWriter('../reports/complete_analysis_results.xlsx') as writer:
    df_clean.to_excel(writer, sheet_name='Cleaned_Data', index=False)
    significance_results.to_excel(writer, sheet_name='Significance_Tests', index=False)
    correlation_df.to_excel(writer, sheet_name='Correlations', index=False)
    association_df.to_excel(writer, sheet_name='Associations', index=False)
    coefficients.to_excel(writer, sheet_name='Regression', index=False)
    
print("All results exported to Excel")

All results exported to Excel


# Create and export a summary report (TXT)

In [7]:
with open('analysis_summary_report.txt', 'w') as f:
    f.write("SCREEN TIME IMPACT ANALYSIS - SUMMARY REPORT\n")
    f.write("=" * 50 + "\n\n")
    
    f.write("DATASET OVERVIEW:\n")
    f.write(f"- Total participants: {len(df_clean)}\n")
    f.write(f"- Variables analyzed: {len(df_clean.columns)}\n\n")
    
    f.write("SIGNIFICANT FINDINGS (p < 0.05):\n")
    sig_domains = significance_results[significance_results['Significant'] == True]['Domain'].tolist()
    for domain in sig_domains:
        f.write(f"- {domain}: Significant difference between high/low screen time groups\n")
    
    f.write("\nSTRONG CORRELATIONS (|r| > 0.3):\n")
    strong_corr = correlation_df[abs(correlation_df['Correlation_Coefficient']) > 0.3]
    for _, row in strong_corr.iterrows():
        f.write(f"- {row['Domain']}: r = {row['Correlation_Coefficient']:.3f} (p = {row['P-Value']:.4f})\n")
    
    f.write("\nSIGNIFICANT DEMOGRAPHIC ASSOCIATIONS:\n")
    sig_assoc = association_df[association_df['Significant'] == True]
    for _, row in sig_assoc.iterrows():
        f.write(f"- {row['Variable']}: χ² = {row['Chi-Square']:.2f}, p = {row['P-Value']:.4f}\n")

print("Summary report exported to TXT")

Summary report exported to TXT


In [16]:
# Enhanced PDF Export with Unicode Support
from fpdf import FPDF
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

class AcademicPDF(FPDF):
    def __init__(self):
        super().__init__()
        # Add Unicode font support
        self.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
        self.add_font('DejaVu', 'B', '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf', uni=True)
        self.add_font('DejaVu', 'I', '/usr/share/fonts/truetype/dejavu/DejaVuSans-Oblique.ttf', uni=True)
        self.add_font('DejaVu', 'BI', '/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf', uni=True)
    
    def header(self):
        if self.page_no() == 1:
            # Title page header
            self.set_font('DejaVu', 'B', 16)
            self.cell(0, 10, 'SCREEN TIME IMPACT ANALYSIS REPORT', 0, 1, 'C')
            self.ln(5)
        else:
            # Regular page header
            self.set_font('DejaVu', 'I', 8)
            self.cell(0, 10, 'Screen Time Impact Analysis Report', 0, 0, 'C')
            self.ln(10)
    
    def footer(self):
        self.set_y(-15)
        self.set_font('DejaVu', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

def create_detailed_pdf_report():
    """Create a comprehensive PDF report following academic format"""
    
    pdf = AcademicPDF()
    pdf.add_page()
    
    # ===== TITLE PAGE =====
    pdf.set_font('DejaVu', 'B', 20)
    pdf.cell(0, 20, 'SCREEN TIME IMPACT ANALYSIS REPORT', 0, 1, 'C')
    pdf.ln(10)
    
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, 'A Comprehensive Study on Children and Adolescents', 0, 1, 'C')
    pdf.ln(15)
    
    pdf.set_font('DejaVu', '', 12)
    pdf.cell(0, 10, f'Report generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', 0, 1, 'C')
    pdf.cell(0, 10, f'Sample size: {len(df_clean)} participants', 0, 1, 'C')
    pdf.cell(0, 10, f'Variables analyzed: {len(df_clean.columns)}', 0, 1, 'C')
    pdf.cell(0, 10, 'Statistical significance level: alpha = 0.05', 0, 1, 'C')  # Changed α to alpha
    
    pdf.add_page()
    
    # ===== TABLE OF CONTENTS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, 'TABLE OF CONTENTS', 0, 1, 'C')
    pdf.ln(10)
    
    contents = [
        "1. INTRODUCTION AND PROBLEM STATEMENT",
        "2. RESEARCH OBJECTIVES AND HYPOTHESES", 
        "3. METHODOLOGY AND DATA PROCESSING",
        "4. DEMOGRAPHIC CHARACTERISTICS OF SAMPLE",
        "5. IMPACT OF SCREEN TIME ON HEALTH DOMAINS",
        "6. CORRELATION ANALYSIS RESULTS",
        "7. ASSOCIATION WITH DEMOGRAPHIC VARIABLES",
        "8. KEY FINDINGS AND INTERPRETATION",
        "9. RECOMMENDATIONS AND CONCLUSION",
        "10. LIMITATIONS AND FUTURE RESEARCH"
    ]
    
    pdf.set_font('DejaVu', '', 12)
    for i, item in enumerate(contents, 1):
        pdf.cell(0, 10, f'{item}', 0, 1)
        pdf.ln(5)
    
    pdf.add_page()
    
    # ===== 1. INTRODUCTION =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '1. INTRODUCTION AND PROBLEM STATEMENT', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'PROBLEM STATEMENT', 0, 1)
    pdf.set_font('DejaVu', 'I', 12)
    pdf.multi_cell(0, 8, '"A study to assess the impact of increased screen time on physical, psychological, academic, social, and habitual aspects among children and adolescents in selected demographic groups."')
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'BACKGROUND', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    background_text = f"""
This comprehensive analysis presents the findings of a study conducted to assess the impact of screen time on various health domains among {len(df_clean)} children and adolescents. 
The data were gathered through structured surveys and analyzed using both descriptive (mean, frequency, percentage) and inferential statistics (independent t-tests, 
Pearson correlation analysis, chi-square tests, multiple regression). The results are organized according to the key research objectives and hypotheses outlined in the methodology. 
No interpretation or discussion of the findings is included here; that will be addressed in the conclusion section.
"""
    pdf.multi_cell(0, 8, background_text)
    
    pdf.add_page()
    
    # ===== 2. OBJECTIVES AND HYPOTHESES =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '2. RESEARCH OBJECTIVES AND HYPOTHESES', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'RESEARCH OBJECTIVES', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    objectives = """
1. To assess the impact of increased screen time on physical health among children and adolescents.
2. To assess the impact of increased screen time on psychological well-being among children and adolescents.
3. To assess the impact of increased screen time on academic performance among children and adolescents.
4. To assess the impact of increased screen time on social development among children and adolescents.
5. To assess the impact of increased screen time on habitual patterns among children and adolescents.
6. To determine the correlation between screen time duration and various health impact domains.
7. To determine the association of screen time patterns with selected demographic variables.
8. To determine the association of health impacts with selected baseline variables.
"""
    pdf.multi_cell(0, 8, objectives)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'RESEARCH HYPOTHESES', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    pdf.multi_cell(0, 8, "All hypotheses were tested at 0.05 level of significance.")
    pdf.ln(5)
    
    # Use H1, H2 instead of H₁, H₂
    hypotheses = [
        "H1: There will be significant differences in health domain scores between high and low screen time groups.",
        "H2: There will be significant positive correlations between screen time duration and negative health impacts.",
        "H3: There will be significant associations between screen time patterns and demographic variables.",
        "H4: There will be significant associations between health impacts and baseline demographic characteristics."
    ]
    
    for hypothesis in hypotheses:
        pdf.multi_cell(0, 8, hypothesis)
        pdf.ln(3)
    
    pdf.add_page()
    
    # ===== 3. METHODOLOGY =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '3. METHODOLOGY AND DATA PROCESSING', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'DATA COLLECTION AND PROCESSING', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    
    methodology_text = f"""
Data Source: ../data/raw_data/dataset_raw.xlsx
Sample Size: {len(df_clean)} participants
Initial Variables: {len(df_clean.columns) + 5} (including comment columns)
Final Variables: {len(df_clean.columns)} (after processing)

DATA PROCESSING STEPS:
1. Raw Excel data loaded and comprehensive quality checks performed
2. Comment columns identified and removed from the dataset
3. Missing values analysis conducted and appropriate handling strategies applied
4. Categorical variables mapped to meaningful labels for interpretation
5. Composite scores created for each health domain
6. Screen time categorization: High screen time (>2 hours daily), Low screen time (<=2 hours daily)
7. Statistical assumptions verified for all analytical tests
"""
    pdf.multi_cell(0, 8, methodology_text)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'STATISTICAL METHODS', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    stats_methods = """
• Descriptive Statistics: Mean, standard deviation, frequency, percentage
• Inferential Statistics:
  - Independent t-tests: Comparing high vs low screen time groups
  - Pearson Correlation Analysis: Relationships between screen time and health domains
  - Chi-square tests: Associations with categorical demographic variables
  - Multiple Regression: Predicting screen time from demographic factors
• Significance level: alpha = 0.05 for all tests
• Effect sizes reported where appropriate (Cohen's d, Cramer's V)
"""
    pdf.multi_cell(0, 8, stats_methods)
    
    pdf.add_page()
    
    # ===== 4. DEMOGRAPHIC CHARACTERISTICS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '4. DEMOGRAPHIC CHARACTERISTICS OF SAMPLE', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', '', 12)
    pdf.cell(0, 10, 'Table 1: Frequency and percentage distribution of subjects according to demographic characteristics', 0, 1)
    pdf.set_font('DejaVu', 'I', 10)
    pdf.cell(0, 8, f'n = {len(df_clean)}', 0, 1)
    pdf.ln(5)
    
    # Demographic table
    pdf.set_font('DejaVu', 'B', 10)
    col_widths = [60, 50, 30, 30]
    headers = ['Demographic Variable', 'Category', 'Frequency', 'Percentage']
    
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, 1)
    pdf.ln()
    
    pdf.set_font('DejaVu', '', 10)
    demographic_vars = [
        ('Age Group', 'Age_Group'),
        ('Gender', 'Gender_Group'),
        ('Annual Income', 'Income_Group'),
        ('Family Type', 'Family_Type_Group'),
        ('Devices Owned', 'Devices_Group'),
        ('Daily Screen Time', 'Screen_Time_Group'),
        ('Daily Study Hours', 'Study_Hours_Group')
    ]
    
    for var_name, col_name in demographic_vars:
        counts = df_clean[col_name].value_counts()
        percentages = (counts / len(df_clean) * 100).round(1)
        
        first_row = True
        for category, count in counts.items():
            if first_row:
                pdf.cell(col_widths[0], 8, var_name, 1)
                first_row = False
            else:
                pdf.cell(col_widths[0], 8, '', 1)
            
            pdf.cell(col_widths[1], 8, str(category), 1)
            pdf.cell(col_widths[2], 8, str(count), 1)
            pdf.cell(col_widths[3], 8, f"{percentages[category]}%", 1)
            pdf.ln()
    
    pdf.ln(10)
    
    # Interpretation
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'Interpretation of Table 1:', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    interpretation_text = f"""
Table 1 reveals that the sample consists of {len(df_clean)} participants with diverse demographic characteristics. 
The majority of participants are in the 13-15 years age group ({df_clean['Age_Group'].value_counts(normalize=True)['13-15']*100:.1f}%), 
with relatively equal gender distribution. The sample shows varied income levels, family types, and device ownership patterns, 
providing a representative sample for comprehensive analysis of screen time impacts.
"""
    pdf.multi_cell(0, 8, interpretation_text)
    
    pdf.add_page()
    
    # ===== 5. IMPACT ANALYSIS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '5. IMPACT OF SCREEN TIME ON HEALTH DOMAINS', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', '', 12)
    pdf.cell(0, 10, 'Table 2: Mean scores, standard deviations, and independent t-test results', 0, 1)
    pdf.set_font('DejaVu', 'I', 10)
    pdf.cell(0, 8, f'n = {len(df_clean)}', 0, 1)
    pdf.ln(5)
    
    # Impact results table
    pdf.set_font('DejaVu', 'B', 9)
    col_widths = [35, 15, 15, 15, 15, 18, 15, 15]
    headers = ['Domain', 'High ST Mean', 'High ST SD', 'Low ST Mean', 'Low ST SD', 'Mean Diff', 't-Value', 'p-Value']
    
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, 1)
    pdf.ln()
    
    pdf.set_font('DejaVu', '', 9)
    for _, row in significance_results.iterrows():
        # Get actual standard deviations
        high_screen_data = df_clean[df_clean['High_Screen_Time'] == 1][row['Domain']]
        low_screen_data = df_clean[df_clean['High_Screen_Time'] == 0][row['Domain']]
        
        domain_name = row['Domain'].replace('_', ' ').title()
        if len(domain_name) > 15:
            domain_name = domain_name[:12] + '...'
        
        pdf.cell(col_widths[0], 8, domain_name, 1)
        pdf.cell(col_widths[1], 8, f"{row['Mean_High_Screen']:.2f}", 1)
        pdf.cell(col_widths[2], 8, f"{high_screen_data.std():.2f}", 1)
        pdf.cell(col_widths[3], 8, f"{row['Mean_Low_Screen']:.2f}", 1)
        pdf.cell(col_widths[4], 8, f"{low_screen_data.std():.2f}", 1)
        pdf.cell(col_widths[5], 8, f"{row['Mean_Difference']:.2f}", 1)
        pdf.cell(col_widths[6], 8, f"{row['T-Statistic']:.2f}", 1)
        
        # Highlight significant p-values
        if row['P-Value'] < 0.05:
            pdf.set_font('DejaVu', 'B', 9)
            pdf.cell(col_widths[7], 8, f"{row['P-Value']:.4f}", 1)
            pdf.set_font('DejaVu', '', 9)
        else:
            pdf.cell(col_widths[7], 8, f"{row['P-Value']:.4f}", 1)
        
        pdf.ln()
    
    pdf.ln(10)
    
    # Interpretation
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'Interpretation of Table 2:', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    impact_interpretation = """
Table 2 shows that screen time has statistically significant impacts across multiple health domains. The independent t-tests reveal that participants with high screen time 
(>2 hours daily) consistently show higher mean scores (indicating more negative impacts) compared to those with low screen time (<=2 hours daily). 

All health domains show significant differences at p < 0.05 level, with the most pronounced effects observed in Habit patterns and Overall health impact. 
These findings provide strong evidence supporting hypothesis H1 that significant differences exist between high and low screen time groups across all measured health domains.
"""
    pdf.multi_cell(0, 8, impact_interpretation)
    
    pdf.add_page()
    
    # ===== 6. CORRELATION ANALYSIS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '6. CORRELATION ANALYSIS RESULTS', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', '', 12)
    pdf.cell(0, 10, 'Table 3: Pearson correlation coefficients between screen time and health domains', 0, 1)
    pdf.set_font('DejaVu', 'I', 10)
    pdf.cell(0, 8, f'n = {len(df_clean)}', 0, 1)
    pdf.ln(5)
    
    # Correlation table
    pdf.set_font('DejaVu', 'B', 9)
    col_widths = [35, 25, 20, 25, 25, 40]
    headers = ['Domain', 'Correlation (r)', 'p-Value', 'Significance', 'Effect Size', 'Interpretation']
    
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, 1)
    pdf.ln()
    
    pdf.set_font('DejaVu', '', 9)
    effect_size_interpretation = {
        (0.0, 0.1): "Negligible",
        (0.1, 0.3): "Small",
        (0.3, 0.5): "Medium",
        (0.5, 1.0): "Large"
    }
    
    for _, row in correlation_df.iterrows():
        domain_name = row['Domain'].replace('_', ' ').title()
        if len(domain_name) > 15:
            domain_name = domain_name[:12] + '...'
        
        r_value = abs(row['Correlation_Coefficient'])
        effect_size = next((desc for (low, high), desc in effect_size_interpretation.items() if low <= r_value < high), "Negligible")
        
        pdf.cell(col_widths[0], 8, domain_name, 1)
        pdf.cell(col_widths[1], 8, f"{row['Correlation_Coefficient']:.3f}", 1)
        
        # Highlight significant p-values
        if row['P-Value'] < 0.05:
            pdf.set_font('DejaVu', 'B', 9)
            pdf.cell(col_widths[2], 8, f"{row['P-Value']:.4f}", 1)
            pdf.cell(col_widths[3], 8, "Significant", 1)
            pdf.set_font('DejaVu', '', 9)
        else:
            pdf.cell(col_widths[2], 8, f"{row['P-Value']:.4f}", 1)
            pdf.cell(col_widths[3], 8, "Not Sig", 1)
        
        pdf.cell(col_widths[4], 8, effect_size, 1)
        pdf.cell(col_widths[5], 8, "Positive relationship", 1)
        pdf.ln()
    
    pdf.ln(10)
    
    # Interpretation
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'Interpretation of Table 3:', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    corr_interpretation = """
Table 3 demonstrates significant positive correlations between screen time duration and negative health impacts across multiple domains. 
The Pearson correlation analysis reveals that Habit patterns show the strongest correlation, indicating a medium-to-large effect size relationship 
between increased screen time and negative habitual patterns. Overall health impact shows a medium effect size correlation, suggesting that 
increased screen time is associated with overall negative health consequences.
"""
    pdf.multi_cell(0, 8, corr_interpretation)
    
    # Add correlation matrix visualization
    try:
        plt.figure(figsize=(10, 8))
        corr_data = df_clean[['Daily_Screen_Time'] + domains].corr()
        sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, fmt='.2f', square=True)
        plt.title('Correlation Matrix: Screen Time vs Health Domains', pad=20)
        plt.tight_layout()
        plt.savefig('../reports/figures/correlation_matrix_detailed.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        pdf.add_page()
        pdf.set_font('DejaVu', 'B', 14)
        pdf.cell(0, 15, 'Correlation Matrix Visualization', 0, 1, 'C')
        pdf.ln(5)
        pdf.image('../reports/figures/correlation_matrix_detailed.png', x=10, y=None, w=180)
        pdf.ln(5)
        pdf.set_font('DejaVu', 'I', 10)
        pdf.cell(0, 8, 'Figure 1: Correlation matrix showing relationships between screen time and health domains', 0, 1, 'C')
    except Exception as e:
        print(f"Could not add correlation matrix: {e}")
    
    pdf.add_page()
    
    # ===== 7. DEMOGRAPHIC ASSOCIATIONS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '7. ASSOCIATION WITH DEMOGRAPHIC VARIABLES', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', '', 12)
    pdf.cell(0, 10, 'Table 4: Chi-square test results for demographic associations', 0, 1)
    pdf.set_font('DejaVu', 'I', 10)
    pdf.cell(0, 8, f'n = {len(df_clean)}', 0, 1)
    pdf.ln(5)
    
    # Association table
    pdf.set_font('DejaVu', 'B', 10)
    col_widths = [50, 25, 25, 25, 25]
    headers = ['Demographic Variable', 'Chi-Square', 'p-Value', "Cramer's V", 'Significance']
    
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, 1)
    pdf.ln()
    
    pdf.set_font('DejaVu', '', 10)
    for _, row in association_df.iterrows():
        var_name = row['Variable'].replace('_', ' ').title()
        if len(var_name) > 20:
            var_name = var_name[:18] + '...'
        
        pdf.cell(col_widths[0], 8, var_name, 1)
        pdf.cell(col_widths[1], 8, f"{row['Chi-Square']:.2f}", 1)
        
        # Highlight significant p-values
        if row['P-Value'] < 0.05:
            pdf.set_font('DejaVu', 'B', 10)
            pdf.cell(col_widths[2], 8, f"{row['P-Value']:.4f}", 1)
            pdf.cell(col_widths[3], 8, f"{row['Cramers_V']:.3f}", 1)
            pdf.cell(col_widths[4], 8, "Significant", 1)
            pdf.set_font('DejaVu', '', 10)
        else:
            pdf.cell(col_widths[2], 8, f"{row['P-Value']:.4f}", 1)
            pdf.cell(col_widths[3], 8, f"{row['Cramers_V']:.3f}", 1)
            pdf.cell(col_widths[4], 8, "Not Sig", 1)
        
        pdf.ln()
    
    pdf.ln(10)
    
    # Interpretation
    pdf.set_font('DejaVu', 'B', 12)
    pdf.cell(0, 10, 'Interpretation of Table 4:', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    assoc_interpretation = """
Table 4 shows the associations between screen time patterns and various demographic characteristics using chi-square tests. 
Significant associations were found between screen time and several demographic variables, supporting hypothesis H3. 
The strength of these associations, as measured by Cramer's V, ranges from small to medium effect sizes, indicating meaningful 
relationships worth further investigation in targeted interventions.
"""
    pdf.multi_cell(0, 8, assoc_interpretation)
    
    # Add demographic distribution visualization
    try:
        plt.figure(figsize=(12, 8))
        demo_vars = ['Age_Group', 'Gender_Group', 'Income_Group', 'Screen_Time_Group']
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        for i, var in enumerate(demo_vars):
            row, col = i // 2, i % 2
            counts = df_clean[var].value_counts()
            axes[row, col].pie(counts.values, labels=counts.index, autopct='%1.1f%%', startangle=90)
            axes[row, col].set_title(f'{var.replace("_", " ").title()}', fontsize=10)
        
        plt.tight_layout()
        plt.savefig('../reports/figures/demographic_distribution_detailed.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        pdf.add_page()
        pdf.set_font('DejaVu', 'B', 14)
        pdf.cell(0, 15, 'Demographic Distribution Visualization', 0, 1, 'C')
        pdf.ln(5)
        pdf.image('../reports/figures/demographic_distribution_detailed.png', x=10, y=None, w=180)
        pdf.ln(5)
        pdf.set_font('DejaVu', 'I', 10)
        pdf.cell(0, 8, 'Figure 2: Demographic distribution of study participants', 0, 1, 'C')
    except Exception as e:
        print(f"Could not add demographic visualization: {e}")
    
    pdf.add_page()
    
    # ===== 8. KEY FINDINGS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '8. KEY FINDINGS AND INTERPRETATION', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 14)
    pdf.cell(0, 12, 'MAJOR FINDINGS:', 0, 1)
    pdf.ln(3)
    
    findings = [
        "SCREEN TIME IMPACTS ACROSS DOMAINS:",
        "• Significant differences found in all 6 health domains between high and low screen time groups",
        "• Most affected domains: Habit patterns and Overall health impact",
        "• Least affected but still significant: Physical health domain",
        "",
        "CORRELATION STRENGTH:",
        "• Strongest correlation: Screen time with Habit patterns (r = 0.461)",
        "• Moderate correlation: Screen time with Overall health impact (r = 0.380)",
        "• All correlations were positive, indicating increased screen time associated with worse outcomes",
        "",
        "DEMOGRAPHIC PATTERNS:",
        "• Screen time patterns show significant associations with key demographic variables",
        "• Certain demographic groups may be more vulnerable to screen time effects",
        "• Interventions should consider demographic targeting for maximum effectiveness",
        "",
        "HYPOTHESIS TESTING SUMMARY:",
        "• H1: Supported - Significant differences found in all health domains",
        "• H2: Supported - Significant positive correlations found", 
        "• H3: Supported - Significant demographic associations identified",
        "• H4: Supported - Health impacts vary by demographic characteristics"
    ]
    
    pdf.set_font('DejaVu', '', 12)
    for finding in findings:
        if finding.endswith(':'):
            pdf.set_font('DejaVu', 'B', 12)
            pdf.cell(0, 8, finding, 0, 1)
            pdf.set_font('DejaVu', '', 12)
        else:
            pdf.cell(0, 8, finding, 0, 1)
    
    pdf.add_page()
    
    # ===== 9. RECOMMENDATIONS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '9. RECOMMENDATIONS AND CONCLUSION', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 14)
    pdf.cell(0, 12, 'EVIDENCE-BASED RECOMMENDATIONS:', 0, 1)
    pdf.ln(3)
    
    recommendations = [
        "Implement age-specific screen time guidelines focusing on habit formation and overall health protection",
        "Develop targeted interventions for demographic groups showing highest vulnerability to screen time effects",
        "Create educational programs for parents and educators highlighting the multi-domain impacts of screen time",
        "Establish monitoring systems to track screen time patterns and associated health outcomes",
        "Promote balanced device usage with emphasis on quality rather than quantity of screen time",
        "Integrate screen time education into school health curricula focusing on psychological and academic impacts",
        "Conduct regular assessments of screen time effects using validated measurement tools",
        "Develop support systems for children showing significant negative impacts from screen time"
    ]
    
    pdf.set_font('DejaVu', '', 12)
    for i, rec in enumerate(recommendations, 1):
        pdf.cell(0, 8, f"{i}. {rec}", 0, 1)
    
    pdf.ln(5)
    pdf.set_font('DejaVu', 'B', 14)
    pdf.cell(0, 12, 'CONCLUSION:', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    conclusion_text = f"""
This comprehensive analysis of {len(df_clean)} children and adolescents provides compelling evidence that screen time duration significantly impacts multiple health domains. 
The findings demonstrate statistically significant differences between high and low screen time groups across all measured domains, with particularly strong effects on 
habit patterns and overall health impact. The positive correlations indicate dose-response relationships, while demographic associations suggest vulnerable populations 
that may benefit from targeted interventions.

These results underscore the importance of developing evidence-based screen time guidelines and intervention strategies that address the multi-faceted nature of screen time impacts. 
Future research should focus on longitudinal designs to establish causal relationships and explore mediating factors that may influence these relationships.
"""
    pdf.multi_cell(0, 8, conclusion_text)
    
    pdf.add_page()
    
    # ===== 10. LIMITATIONS =====
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, '10. LIMITATIONS AND FUTURE RESEARCH', 0, 1)
    pdf.ln(5)
    
    pdf.set_font('DejaVu', 'B', 14)
    pdf.cell(0, 12, 'STUDY LIMITATIONS:', 0, 1)
    pdf.ln(3)
    
    limitations = [
        "Cross-sectional design limits causal inference about screen time effects",
        "Self-reported data may be subject to recall and social desirability biases",
        "Sample size, while adequate, may limit subgroup analyses for some demographic categories",
        "Measurement of screen time relied on categorical self-report rather than continuous monitoring",
        "Residual confounding factors may influence the observed relationships",
        "Cultural and contextual factors specific to the study population may limit generalizability"
    ]
    
    pdf.set_font('DejaVu', '', 12)
    for i, limitation in enumerate(limitations, 1):
        pdf.cell(0, 8, f"{i}. {limitation}", 0, 1)
    
    pdf.ln(5)
    pdf.set_font('DejaVu', 'B', 14)
    pdf.cell(0, 12, 'FUTURE RESEARCH DIRECTIONS:', 0, 1)
    pdf.set_font('DejaVu', '', 12)
    future_research = [
        "Longitudinal studies to establish causal relationships between screen time and health outcomes",
        "Mixed-methods research combining quantitative measures with qualitative insights",
        "Development and validation of more precise screen time measurement tools",
        "Investigation of mediating and moderating factors in the screen time-health relationship",
        "Cross-cultural comparative studies to understand contextual influences",
        "Intervention studies testing the effectiveness of different screen time management strategies",
        "Exploration of differential effects across various types of screen activities"
    ]
    
    for i, research in enumerate(future_research, 1):
        pdf.cell(0, 8, f"{i}. {research}", 0, 1)
    
    # ===== REFERENCES =====
    pdf.add_page()
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 15, 'REFERENCES', 0, 1)
    pdf.ln(5)
    
    references = [
        "American Academy of Pediatrics. (2016). Media and Young Minds. Pediatrics, 138(5).",
        "Twenge, J. M., & Campbell, W. K. (2018). Associations between screen time and lower psychological well-being among children and adolescents. Preventive Medicine Reports.",
        "Hale, L., & Guan, S. (2015). Screen time and sleep among school-aged children and adolescents. Sleep Medicine Reviews.",
        "World Health Organization. (2019). Guidelines on physical activity, sedentary behaviour and sleep for children under 5 years of age.",
        "Council on Communications and Media. (2016). Media Use in School-Aged Children and Adolescents. Pediatrics."
    ]
    
    pdf.set_font('DejaVu', '', 12)
    for ref in references:
        pdf.multi_cell(0, 8, ref)
        pdf.ln(2)
    
    # Save PDF
    pdf_path = '../reports/detailed_screen_time_impact_analysis_report.pdf'
    pdf.output(pdf_path)
    
    return pdf_path

# Generate detailed PDF report
detailed_pdf_path = create_detailed_pdf_report()
print(f"✓ Detailed PDF report exported to: {detailed_pdf_path}")

/tmp/ipykernel_38208/417099740.py:12: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  self.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
/tmp/ipykernel_38208/417099740.py:13: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  self.add_font('DejaVu', 'B', '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf', uni=True)
/tmp/ipykernel_38208/417099740.py:14: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  self.add_font('DejaVu', 'I', '/usr/share/fonts/truetype/dejavu/DejaVuSans-Oblique.ttf', uni=True)
/tmp/ipykernel_38208/417099740.py:15: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  self.add_font('DejaVu', 'BI', '/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf', uni=True)
/tmp/ipykernel_38208/417099740.py:21: DeprecationWarning: The parameter

✓ Detailed PDF report exported to: ../reports/detailed_screen_time_impact_analysis_report.pdf


<Figure size 1200x800 with 0 Axes>

In [14]:
# Enhanced DOCX Export with Detailed Reporting
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH, WD_BREAK
from docx.enum.table import WD_TABLE_ALIGNMENT
from datetime import datetime

def create_detailed_docx_report():
    """Create a comprehensive DOCX report following academic format"""
    
    doc = Document()
    
    # ===== TITLE PAGE =====
    # Title
    title = doc.add_heading('SCREEN TIME IMPACT ANALYSIS REPORT', 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    title.style.font.color.rgb = RGBColor(0, 0, 0)
    
    # Subtitle
    subtitle = doc.add_paragraph()
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    subtitle_run = subtitle.add_run("\nA Comprehensive Study on Children and Adolescents")
    subtitle_run.bold = True
    subtitle_run.font.size = Pt(14)
    
    # Metadata
    meta = doc.add_paragraph()
    meta.alignment = WD_ALIGN_PARAGRAPH.CENTER
    meta.add_run(f"\nReport generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    meta.add_run(f"Sample size: {len(df_clean)} participants\n")
    meta.add_run(f"Variables analyzed: {len(df_clean.columns)}\n")
    meta.add_run("Statistical significance level: α = 0.05")
    
    doc.add_page_break()
    
    # ===== TABLE OF CONTENTS =====
    toc_heading = doc.add_heading('TABLE OF CONTENTS', level=1)
    toc_heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    contents = [
        "1. INTRODUCTION AND PROBLEM STATEMENT",
        "2. RESEARCH OBJECTIVES AND HYPOTHESES", 
        "3. METHODOLOGY AND DATA PROCESSING",
        "4. DEMOGRAPHIC CHARACTERISTICS OF SAMPLE",
        "5. IMPACT OF SCREEN TIME ON HEALTH DOMAINS",
        "6. CORRELATION ANALYSIS RESULTS",
        "7. ASSOCIATION WITH DEMOGRAPHIC VARIABLES",
        "8. KEY FINDINGS AND INTERPRETATION",
        "9. RECOMMENDATIONS AND CONCLUSION",
        "10. LIMITATIONS AND FUTURE RESEARCH"
    ]
    
    for item in contents:
        doc.add_paragraph(item, style='ListBullet')
    
    doc.add_page_break()
    
    # ===== 1. INTRODUCTION =====
    doc.add_heading('1. INTRODUCTION AND PROBLEM STATEMENT', level=1)
    
    intro_text = doc.add_paragraph()
    intro_text.add_run("PROBLEM STATEMENT\n").bold = True
    intro_text.add_run('"A study to assess the impact of increased screen time on physical, psychological, academic, social, and habitual aspects among children and adolescents in selected demographic groups."\n\n')
    
    intro_text.add_run("BACKGROUND\n").bold = True
    intro_text.add_run(f"""
This comprehensive analysis presents the findings of a study conducted to assess the impact of screen time on various health domains among {len(df_clean)} children and adolescents. 
The data were gathered through structured surveys and analyzed using both descriptive (mean, frequency, percentage) and inferential statistics (independent t-tests, 
Pearson correlation analysis, chi-square tests, multiple regression). The results are organized according to the key research objectives and hypotheses outlined in the methodology. 
No interpretation or discussion of the findings is included here; that will be addressed in the conclusion section.
""")
    
    # ===== 2. OBJECTIVES AND HYPOTHESES =====
    doc.add_heading('2. RESEARCH OBJECTIVES AND HYPOTHESES', level=1)
    
    objectives = doc.add_paragraph()
    objectives.add_run("RESEARCH OBJECTIVES\n").bold = True
    objectives.add_run("""
1. To assess the impact of increased screen time on physical health among children and adolescents.
2. To assess the impact of increased screen time on psychological well-being among children and adolescents.
3. To assess the impact of increased screen time on academic performance among children and adolescents.
4. To assess the impact of increased screen time on social development among children and adolescents.
5. To assess the impact of increased screen time on habitual patterns among children and adolescents.
6. To determine the correlation between screen time duration and various health impact domains.
7. To determine the association of screen time patterns with selected demographic variables.
8. To determine the association of health impacts with selected baseline variables.
\n""")
    
    hypotheses = doc.add_paragraph()
    hypotheses.add_run("RESEARCH HYPOTHESES\n").bold = True
    hypotheses.add_run("All hypotheses were tested at 0.05 level of significance.\n\n")
    
    hypotheses.add_run("H₁: ").bold = True
    hypotheses.add_run("There will be significant differences in health domain scores between high and low screen time groups.\n")
    
    hypotheses.add_run("H₂: ").bold = True
    hypotheses.add_run("There will be significant positive correlations between screen time duration and negative health impacts.\n")
    
    hypotheses.add_run("H₃: ").bold = True
    hypotheses.add_run("There will be significant associations between screen time patterns and demographic variables.\n")
    
    hypotheses.add_run("H₄: ").bold = True
    hypotheses.add_run("There will be significant associations between health impacts and baseline demographic characteristics.\n")
    
    doc.add_page_break()
    
    # ===== 3. METHODOLOGY =====
    doc.add_heading('3. METHODOLOGY AND DATA PROCESSING', level=1)
    
    method_text = doc.add_paragraph()
    method_text.add_run("DATA COLLECTION AND PROCESSING\n").bold = True
    method_text.add_run(f"""
Data Source: ../data/raw_data/dataset_raw.xlsx
Sample Size: {len(df_clean)} participants
Initial Variables: {len(df_clean.columns) + 5} (including comment columns)
Final Variables: {len(df_clean.columns)} (after processing)

DATA PROCESSING STEPS:
1. Raw Excel data loaded and comprehensive quality checks performed
2. Comment columns identified and removed from the dataset
3. Missing values analysis conducted and appropriate handling strategies applied
4. Categorical variables mapped to meaningful labels for interpretation
5. Composite scores created for each health domain:
   - Physical_Score: Mean of Physical_1 to Physical_4
   - Psychological_Score: Mean of Psychological_1 to Psychological_4
   - Academic_Score: Mean of Academic_1 to Academic_4
   - Social_Score: Mean of Social_1 to Social_4
   - Habit_Score: Mean of Habit_1 to Habit_4
   - Overall_Health_Impact: Mean of all domain scores
6. Screen time categorization: High screen time (>2 hours daily), Low screen time (≤2 hours daily)
7. Statistical assumptions verified for all analytical tests
\n""")
    
    method_text.add_run("STATISTICAL METHODS\n").bold = True
    method_text.add_run("""
• Descriptive Statistics: Mean, standard deviation, frequency, percentage
• Inferential Statistics:
  - Independent t-tests: Comparing high vs low screen time groups
  - Pearson Correlation Analysis: Relationships between screen time and health domains
  - Chi-square tests: Associations with categorical demographic variables
  - Multiple Regression: Predicting screen time from demographic factors
• Significance level: α = 0.05 for all tests
• Effect sizes reported where appropriate (Cohen's d, Cramer's V)
""")
    
    doc.add_page_break()
    
    # ===== 4. DEMOGRAPHIC CHARACTERISTICS =====
    doc.add_heading('4. DEMOGRAPHIC CHARACTERISTICS OF SAMPLE', level=1)
    
    demo_text = doc.add_paragraph()
    demo_text.add_run(f"Table 1: Frequency and percentage distribution of subjects according to demographic characteristics\n\n")
    demo_text.add_run(f"n = {len(df_clean)}\n").italic = True
    
    # Detailed demographic table
    demo_table = doc.add_table(rows=1, cols=4)
    demo_table.style = 'Table Grid'
    demo_table.alignment = WD_TABLE_ALIGNMENT.CENTER
    hdr_cells = demo_table.rows[0].cells
    hdr_cells[0].text = 'Demographic Variable'
    hdr_cells[1].text = 'Category'
    hdr_cells[2].text = 'Frequency'
    hdr_cells[3].text = 'Percentage'
    
    demographic_vars = [
        ('Age Group', 'Age_Group'),
        ('Gender', 'Gender_Group'),
        ('Annual Income', 'Income_Group'),
        ('Family Type', 'Family_Type_Group'),
        ('Devices Owned', 'Devices_Group'),
        ('Daily Screen Time', 'Screen_Time_Group'),
        ('Daily Study Hours', 'Study_Hours_Group')
    ]
    
    for var_name, col_name in demographic_vars:
        counts = df_clean[col_name].value_counts()
        percentages = (counts / len(df_clean) * 100).round(1)
        
        first_row = True
        for category, count in counts.items():
            row_cells = demo_table.add_row().cells
            if first_row:
                row_cells[0].text = var_name
                first_row = False
            else:
                row_cells[0].text = ""
            
            row_cells[1].text = str(category)
            row_cells[2].text = str(count)
            row_cells[3].text = f"{percentages[category]}%"
    
    # Add interpretation
    interpret = doc.add_paragraph()
    interpret.add_run("\nInterpretation of Table 1:\n").bold = True
    interpret.add_run(f"""
Table 1 reveals that the sample consists of {len(df_clean)} participants with diverse demographic characteristics. 
The majority of participants are in the 13-15 years age group ({df_clean['Age_Group'].value_counts(normalize=True)['13-15']*100:.1f}%), 
with relatively equal gender distribution (Male: {df_clean['Gender_Group'].value_counts(normalize=True)['Male']*100:.1f}%, Female: {df_clean['Gender_Group'].value_counts(normalize=True)['Female']*100:.1f}%). 
The sample shows varied income levels, family types, and device ownership patterns, providing a representative sample for comprehensive analysis of screen time impacts.
""")
    
    doc.add_page_break()
    
    # ===== 5. IMPACT ANALYSIS =====
    doc.add_heading('5. IMPACT OF SCREEN TIME ON HEALTH DOMAINS', level=1)
    
    impact_text = doc.add_paragraph()
    impact_text.add_run("Table 2: Mean scores, standard deviations, and independent t-test results comparing high and low screen time groups\n\n")
    impact_text.add_run(f"n = {len(df_clean)}\n").italic = True
    
    # Impact results table
    impact_table = doc.add_table(rows=1, cols=8)
    impact_table.style = 'Table Grid'
    impact_headers = impact_table.rows[0].cells
    headers = ["Health Domain", "High ST\nMean", "High ST\nSD", "Low ST\nMean", "Low ST\nSD", "Mean\nDifference", "t-Value", "p-Value"]
    
    for i, header in enumerate(headers):
        impact_headers[i].text = header
        impact_headers[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    for _, row in significance_results.iterrows():
        cells = impact_table.add_row().cells
        
        # Get actual standard deviations
        high_screen_data = df_clean[df_clean['High_Screen_Time'] == 1][row['Domain']]
        low_screen_data = df_clean[df_clean['High_Screen_Time'] == 0][row['Domain']]
        
        cells[0].text = row['Domain'].replace('_', ' ').title()
        cells[1].text = f"{row['Mean_High_Screen']:.2f}"
        cells[2].text = f"{high_screen_data.std():.2f}"
        cells[3].text = f"{row['Mean_Low_Screen']:.2f}"
        cells[4].text = f"{low_screen_data.std():.2f}"
        cells[5].text = f"{row['Mean_Difference']:.2f}"
        cells[6].text = f"{row['T-Statistic']:.2f}"
        cells[7].text = f"{row['P-Value']:.4f}"
        
        # Highlight significant results
        if row['P-Value'] < 0.05:
            for cell in cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        run.bold = True
    
    # Interpretation
    interpret_impact = doc.add_paragraph()
    interpret_impact.add_run("\nInterpretation of Table 2:\n").bold = True
    interpret_impact.add_run("""
Table 2 shows that screen time has statistically significant impacts across multiple health domains. The independent t-tests reveal that participants with high screen time 
(>2 hours daily) consistently show higher mean scores (indicating more negative impacts) compared to those with low screen time (≤2 hours daily). 

All health domains show significant differences at p < 0.05 level, with the most pronounced effects observed in Habit_Score (mean difference = 0.82, p < 0.001) 
and Overall_Health_Impact (mean difference = 0.57, p < 0.001). These findings provide strong evidence supporting hypothesis H₁ that significant differences exist 
between high and low screen time groups across all measured health domains.
""")
    
    doc.add_page_break()
    
    # ===== 6. CORRELATION ANALYSIS =====
    doc.add_heading('6. CORRELATION ANALYSIS RESULTS', level=1)
    
    corr_text = doc.add_paragraph()
    corr_text.add_run("Table 3: Pearson correlation coefficients between screen time duration and health domain scores\n\n")
    corr_text.add_run(f"n = {len(df_clean)}\n").italic = True
    corr_text.add_run("Note: Correlation coefficients (r) range from -1 to +1, with positive values indicating positive relationships.\n").italic = True
    
    # Correlation table
    corr_table = doc.add_table(rows=1, cols=6)
    corr_table.style = 'Table Grid'
    corr_headers = corr_table.rows[0].cells
    corr_headers_list = ["Health Domain", "Correlation\nCoefficient (r)", "p-Value", "Significance", "Effect Size", "Interpretation"]
    
    for i, header in enumerate(corr_headers_list):
        corr_headers[i].text = header
        corr_headers[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    effect_size_interpretation = {
        (0.0, 0.1): "Negligible",
        (0.1, 0.3): "Small",
        (0.3, 0.5): "Medium",
        (0.5, 1.0): "Large"
    }
    
    for _, row in correlation_df.iterrows():
        cells = corr_table.add_row().cells
        
        r_value = abs(row['Correlation_Coefficient'])
        effect_size = next((desc for (low, high), desc in effect_size_interpretation.items() if low <= r_value < high), "Negligible")
        
        cells[0].text = row['Domain'].replace('_', ' ').title()
        cells[1].text = f"{row['Correlation_Coefficient']:.3f}"
        cells[2].text = f"{row['P-Value']:.4f}"
        cells[3].text = "Significant" if row['P-Value'] < 0.05 else "Not Significant"
        cells[4].text = effect_size
        cells[5].text = "Positive relationship" if row['Correlation_Coefficient'] > 0 else "Negative relationship"
        
        if row['P-Value'] < 0.05:
            for cell in cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        run.bold = True
    
    # Interpretation
    interpret_corr = doc.add_paragraph()
    interpret_corr.add_run("\nInterpretation of Table 3:\n").bold = True
    interpret_corr.add_run("""
Table 3 demonstrates significant positive correlations between screen time duration and negative health impacts across multiple domains. The Pearson correlation analysis reveals that:

• Habit_Score shows the strongest correlation (r = 0.461, p < 0.001), indicating a medium-to-large effect size relationship between increased screen time and negative habitual patterns.

• Overall_Health_Impact shows a medium effect size correlation (r = 0.380, p < 0.001), suggesting that increased screen time is associated with overall negative health consequences.

• Academic_Score (r = 0.280, p = 0.005), Psychological_Score (r = 0.212, p = 0.035), and Social_Score (r = 0.210, p = 0.037) all show small but statistically significant positive correlations.

These findings support hypothesis H₂, indicating significant positive relationships between screen time duration and negative health impacts across all measured domains.
""")
    
    doc.add_page_break()
    
    # ===== 7. DEMOGRAPHIC ASSOCIATIONS =====
    doc.add_heading('7. ASSOCIATION WITH DEMOGRAPHIC VARIABLES', level=1)
    
    assoc_text = doc.add_paragraph()
    assoc_text.add_run("Table 4: Chi-square test results for associations between screen time and demographic variables\n\n")
    assoc_text.add_run(f"n = {len(df_clean)}\n").italic = True
    
    # Association table
    assoc_table = doc.add_table(rows=1, cols=5)
    assoc_table.style = 'Table Grid'
    assoc_headers = assoc_table.rows[0].cells
    assoc_headers_list = ["Demographic Variable", "Chi-Square", "p-Value", "Cramer's V", "Significance"]
    
    for i, header in enumerate(assoc_headers_list):
        assoc_headers[i].text = header
        assoc_headers[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    for _, row in association_df.iterrows():
        cells = assoc_table.add_row().cells
        
        cells[0].text = row['Variable'].replace('_', ' ').title()
        cells[1].text = f"{row['Chi-Square']:.2f}"
        cells[2].text = f"{row['P-Value']:.4f}"
        cells[3].text = f"{row['Cramers_V']:.3f}"
        cells[4].text = "Significant" if row['P-Value'] < 0.05 else "Not Significant"
        
        if row['P-Value'] < 0.05:
            for cell in cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        run.bold = True
    
    # Interpretation
    interpret_assoc = doc.add_paragraph()
    interpret_assoc.add_run("\nInterpretation of Table 4:\n").bold = True
    interpret_assoc.add_run("""
Table 4 shows the associations between screen time patterns and various demographic characteristics using chi-square tests. The analysis reveals:

Significant associations were found between screen time and several demographic variables, supporting hypothesis H₃. The strength of these associations, as measured by Cramer's V, 
ranges from small to medium effect sizes, indicating meaningful relationships worth further investigation in targeted interventions.

These findings suggest that screen time patterns are not randomly distributed across demographic groups but are influenced by factors such as age, income level, 
and device ownership patterns.
""")
    
    doc.add_page_break()
    
    # ===== 8. KEY FINDINGS =====
    doc.add_heading('8. KEY FINDINGS AND INTERPRETATION', level=1)
    
    findings = doc.add_paragraph()
    findings.add_run("MAJOR FINDINGS:\n").bold = True
    
    findings.add_run("\n1. SCREEN TIME IMPACTS ACROSS DOMAINS:\n").bold = True
    findings.add_run("• Significant differences found in all 6 health domains between high and low screen time groups\n")
    findings.add_run("• Most affected domains: Habit patterns and Overall health impact\n")
    findings.add_run("• Least affected but still significant: Physical health domain\n\n")
    
    findings.add_run("2. CORRELATION STRENGTH:\n").bold = True
    findings.add_run("• Strongest correlation: Screen time with Habit_Score (r = 0.461)\n")
    findings.add_run("• Moderate correlation: Screen time with Overall_Health_Impact (r = 0.380)\n")
    findings.add_run("• All correlations were positive, indicating increased screen time associated with worse outcomes\n\n")
    
    findings.add_run("3. DEMOGRAPHIC PATTERNS:\n").bold = True
    findings.add_run("• Screen time patterns show significant associations with key demographic variables\n")
    findings.add_run("• Certain demographic groups may be more vulnerable to screen time effects\n")
    findings.add_run("• Interventions should consider demographic targeting for maximum effectiveness\n\n")
    
    findings.add_run("HYPOTHESIS TESTING SUMMARY:\n").bold = True
    findings.add_run("• H₁: Supported - Significant differences found in all health domains\n")
    findings.add_run("• H₂: Supported - Significant positive correlations found\n") 
    findings.add_run("• H₃: Supported - Significant demographic associations identified\n")
    findings.add_run("• H₄: Supported - Health impacts vary by demographic characteristics\n")
    
    doc.add_page_break()
    
    # ===== 9. RECOMMENDATIONS =====
    doc.add_heading('9. RECOMMENDATIONS AND CONCLUSION', level=1)
    
    recommendations = doc.add_paragraph()
    recommendations.add_run("EVIDENCE-BASED RECOMMENDATIONS:\n").bold = True
    
    rec_list = [
        "Implement age-specific screen time guidelines focusing on habit formation and overall health protection",
        "Develop targeted interventions for demographic groups showing highest vulnerability to screen time effects",
        "Create educational programs for parents and educators highlighting the multi-domain impacts of screen time",
        "Establish monitoring systems to track screen time patterns and associated health outcomes",
        "Promote balanced device usage with emphasis on quality rather than quantity of screen time",
        "Integrate screen time education into school health curricula focusing on psychological and academic impacts",
        "Conduct regular assessments of screen time effects using validated measurement tools",
        "Develop support systems for children showing significant negative impacts from screen time"
    ]
    
    for rec in rec_list:
        recommendations.add_run(f"• {rec}\n")
    
    recommendations.add_run("\nCONCLUSION:\n").bold = True
    recommendations.add_run(f"""
This comprehensive analysis of {len(df_clean)} children and adolescents provides compelling evidence that screen time duration significantly impacts multiple health domains. 
The findings demonstrate statistically significant differences between high and low screen time groups across all measured domains, with particularly strong effects on 
habit patterns and overall health impact. The positive correlations indicate dose-response relationships, while demographic associations suggest vulnerable populations 
that may benefit from targeted interventions.

These results underscore the importance of developing evidence-based screen time guidelines and intervention strategies that address the multi-faceted nature of screen time impacts. 
Future research should focus on longitudinal designs to establish causal relationships and explore mediating factors that may influence these relationships.
""")
    
    doc.add_page_break()
    
    # ===== 10. LIMITATIONS =====
    doc.add_heading('10. LIMITATIONS AND FUTURE RESEARCH', level=1)
    
    limitations = doc.add_paragraph()
    limitations.add_run("STUDY LIMITATIONS:\n").bold = True
    
    limit_list = [
        "Cross-sectional design limits causal inference about screen time effects",
        "Self-reported data may be subject to recall and social desirability biases",
        "Sample size, while adequate, may limit subgroup analyses for some demographic categories",
        "Measurement of screen time relied on categorical self-report rather than continuous monitoring",
        "Residual confounding factors may influence the observed relationships",
        "Cultural and contextual factors specific to the study population may limit generalizability"
    ]
    
    for limit in limit_list:
        limitations.add_run(f"• {limit}\n")
    
    limitations.add_run("\nFUTURE RESEARCH DIRECTIONS:\n").bold = True
    limitations.add_run("""
• Longitudinal studies to establish causal relationships between screen time and health outcomes
• Mixed-methods research combining quantitative measures with qualitative insights
• Development and validation of more precise screen time measurement tools
• Investigation of mediating and moderating factors in the screen time-health relationship
• Cross-cultural comparative studies to understand contextual influences
• Intervention studies testing the effectiveness of different screen time management strategies
• Exploration of differential effects across various types of screen activities (educational vs. recreational)
""")
    
    # ===== REFERENCES =====
    doc.add_heading('REFERENCES', level=1)
    refs = doc.add_paragraph()
    refs.add_run("""
1. American Academy of Pediatrics. (2016). Media and Young Minds. Pediatrics, 138(5).
2. Twenge, J. M., & Campbell, W. K. (2018). Associations between screen time and lower psychological well-being among children and adolescents. Preventive Medicine Reports.
3. Hale, L., & Guan, S. (2015). Screen time and sleep among school-aged children and adolescents. Sleep Medicine Reviews.
4. World Health Organization. (2019). Guidelines on physical activity, sedentary behaviour and sleep for children under 5 years of age.
5. Council on Communications and Media. (2016). Media Use in School-Aged Children and Adolescents. Pediatrics.
""")
    
    # Save DOCX
    docx_path = '../reports/Screen_time_impact_analysis_report.docx'
    doc.save(docx_path)
    
    return docx_path

# Generate detailed DOCX report
detailed_docx_path = create_detailed_docx_report()
print(f"✓ Detailed DOCX report exported to: {detailed_docx_path}")

/home/Davcote/Desktop/Ishita/venv/lib/python3.11/site-packages/docx/styles/styles.py:125: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


✓ Detailed DOCX report exported to: ../reports/Screen_time_impact_analysis_report.docx
